## p-y for sand

# Notes

Need to run the code below again to reinstall the package

```shell
pip install -e . 
```

The code has been moved to the source folder for house keeping

In [8]:
from src.cpt import CPT
from pathlib import Path
import os
import plotly.io as pio
import numpy as np
import pandas as pd
from numpy import degrees, log10, pi, radians, tan, sin, cos
from src.pile import PipePile
import unittest
from src.pyunittests import *
import plotly.graph_objs as go

### Import data from CPT file
#### - Data format - 

`key` is the identifier in the txt file, for locating the data, it shoudl be a unit identifier just above the data
`patter` - an regular experss to specify the pattern of the data, you can see the example below

After reading the data, need to change the column name, so that all calculation can know the location of the data

#### - Rename Columns - 
```python
cpt_data.update_data_column_name(['Reading','SCPT_DPTH','SCPT_RES','SCPT_FRES','SCPT_PWP2','slopex','slopey'])
```

- `SCPT_DPTH` - depth below seabed
- `SCPT_RES` - raw `qc` value
- `SCPT_FRES` - shaft resistance $f_s$
- `SCPT_RES` - tip resistance $q_c$

#### - Specify Unit - 

Need to update the unit, differnet contactor may output data in different unit, this is one by 

```python
cpt_data.set_data_unit(['MPa','MPa','MPa'])
```

unit is assigned in a sequence [$q_c$, $f_s$,$u_2$]

#### - init_CPT - 

This is calculate $\gamma_{soil}$ and calculate the overburnden stress $\sigma_v$ 

The missing values in the original CPT file at initial depth, such as `SCPT_RES`, `SCPT_FRES`, and `SCPT_PWP2`, are backfilled to ensure robustness of the code. 


In [9]:
filename ='CPT4.A00'
fileloc = Path(r'G:\Name Folders\Current Staff\CC213\10 IiA funding\Unified CPT Methods\01 Data\CPT Data\18 0091 05 R002 CPT ASCII')

cpt_data = CPT()
cpt_data.read_ASCII(fileloc / filename,key='Data--',pattern= r'(.{7})(.{10})(.{11})(.{11})(.{11})(.{11})(.{11})')
cpt_data.update_data_column_name(['Reading','SCPT_DPTH','SCPT_RES','SCPT_FRES','SCPT_PWP2','slopex','slopey'])
cpt_data.set_data_unit(['MPa','MPa','MPa'])
cpt_data.init_CPT()
cpt_data.df.to_excel("cpt_data_original.xlsx")

23-11-02 16:46:06 -c:\Users\Emma.Shi\Desktop\CPT\UnitCPT\src\cpt.py:350 DEBUG - Data unit is set to ['MPa', 'MPa', 'MPa']


c:\Users\Emma.Shi\Desktop\CPT\UnitCPT\src\cpt.py:614: RuntimeWarning:

invalid value encountered in log10

c:\Users\Emma.Shi\Desktop\CPT\UnitCPT\src\cpt.py:614: RuntimeWarning:

invalid value encountered in log10

23-11-02 16:46:07 -c:\Users\Emma.Shi\Desktop\CPT\UnitCPT\src\cpt.py:175 DEBUG - qt calculated using net area ratio of 0.85


### Identify soil layers

In [10]:
cpt_data.identify_soil_layers(num_layers = 15, plot_fig = True)

### Input pile information

In [11]:
pile = PipePile(dia = 3.5,thickness = 0.07,length = 70, penetration = 70)

### Calculate bearing capacity

In [18]:
cpt_data.identify_soil_layers(num_layers = 50, plot_fig = False)
cpt_data.calc_pile_capacity(pile, compression = True, plot_fig = True)
cpt_data.df.to_excel("cpt_data_bearing_capacity.xlsx")

TypeError: identify_soil_layers() got an unexpected keyword argument 'plot_fig'

### Calculate p-y curves

In [16]:
cpt_data.calc_p_y_curve(pile, compression = True, monotonic = True, isotropy = True, interval = 1.0, y_range = 800, plot_fig = True, Clay_type = Clay_type.Gulf_of_Mexico)

cpt_data_p_y_curve.xlsx has been exported successfully.


,SCPT_DPTH,SCPT_RES,SCPT_FRES,SCPT_PWP2,slopex,slopey,qt,Rf,gamma,sigma_v,...,y7,p7,y8,p8,y9,p9,y10,p10,y11,p11
0,1.0,0.082000,0.700000,30.275000,6.465212,5.551075,0.086541,0.808869,11.627391,11.231657,...,273.000000,60.219811,420.000000,68.822641,665.000000,77.425471,1050.000000,83.877594,1400.000000,86.028301
1,2.0,0.171018,1.500000,51.454545,6.451691,5.657791,0.178736,0.839923,12.780863,23.614445,...,273.000000,142.644725,420.000000,163.022543,665.000000,183.400361,1050.000000,198.683724,1400.000000,203.778179
2,3.0,0.148725,3.794915,58.466102,6.899124,5.077425,0.157495,2.471774,13.780786,37.152672,...,252.706628,141.583708,388.236461,161.809952,620.883974,182.036196,988.237563,197.205879,1311.767948,202.262440
3,4.0,0.158970,4.570000,81.970000,7.156760,4.514450,0.171266,2.668466,14.044807,50.748322,...,207.653932,164.279867,317.719197,187.748419,522.943330,211.216972,851.120662,228.818386,1115.886660,234.685524
4,5.0,0.193600,5.600000,94.700000,7.481300,3.844700,0.207805,2.694834,14.352510,64.858500,...,195.664013,211.147399,298.952369,241.311313,496.878290,271.475227,814.629606,294.098163,1063.756580,301.639142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,69.0,19.443740,104.900000,631.540000,-1.605880,12.154440,19.538471,0.536902,19.460095,1235.191482,...,75.818670,79620.346146,155.981175,99478.957285,320.898891,102220.657802,660.182857,102250.852473,1358.189190,102250.855164
69,70.0,19.600520,87.910000,666.420000,0.793010,12.083570,19.700483,0.446239,19.260283,1254.599320,...,75.818670,79491.074100,155.981175,98465.494240,320.898891,100940.705329,660.182857,100965.389259,1358.189190,100965.391061
70,71.0,24.984230,130.380000,619.610000,3.280050,11.500970,25.077171,0.519932,19.805601,1273.939141,...,75.818670,97529.195404,155.981175,122024.438037,320.898891,125436.962266,660.182857,125475.142045,1358.189190,125475.145557
71,72.0,20.591850,92.050000,629.300000,5.460900,10.575200,20.686245,0.445048,19.331491,1293.503624,...,75.818670,82907.538282,155.981175,103005.961428,320.898891,105680.529933,660.182857,105708.149524,1358.189190,105708.151687


### Unittests

In [14]:
# Create a TestSuite object
suite = unittest.TestSuite()

# Add the test_p_y_sand_monotonic method to the TestSuite
suite.addTest(TestUnifiedCPT('test_p_y_sand_monotonic'))
suite.addTest(TestUnifiedCPT('test_p_y_sand_cyclic'))
suite.addTest(TestUnifiedCPT('test_p_y_clay_monotonic'))

# Create a TestRunner object
runner = unittest.TextTestRunner()

# Run the TestSuite using the TestRunner
runner.run(suite)

...
----------------------------------------------------------------------
Ran 3 tests in 0.006s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

In [15]:
import dash
from dash import html, dcc
from dash.dependencies import Input, Output

# Create the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1('My Dashboard'),
    dcc.Dropdown(
        id='column-dropdown',
        options=[{'label': col, 'value': col} for col in resampled_cpt_data.columns],
        value=resampled_cpt_data.columns[0]
    ),
    dcc.Graph(id='my-graph')
])

# Define the callback function that updates the graph
@app.callback(
    Output('my-graph', 'figure'),
    [Input('column-dropdown', 'value')]
)
def update_graph(column):
    # Create a scatter plot trace with the selected column and depth
    trace = dict(
        x=resampled_cpt_data['SCPT_DPTH'],
        y=resampled_cpt_data[column],
        mode='lines+markers',
        marker=dict(size=10, color='blue', opacity=0.5)
    )

    # Create the figure object and return it
    fig = dict(data=[trace])
    fig['layout'] = dict(
        xaxis=dict(title='Depth (m)'),
        yaxis=dict(title=column)
    )
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

ImportError: cannot import name '_plain_int' from 'werkzeug._internal' (c:\ProgramData\Anaconda3\envs\dcpyTutorial\lib\site-packages\werkzeug\_internal.py)

# Sampling the P-y points

** To-Do** 

[ ]Need to read original paper to workout the points. 

Appendix `A.8.5.2.1.3` provides more detailed discussion on the rational of determining the p-y points. It will be more elegant to implement the p-y curves for clay in this way

The p-y points are based on scaling the DSS stress-strain curves as follows:

$$
\begin{equation}
\frac{\tau}{s_u} = \frac{\tanh\bigg(a \cdot (\frac{\gamma}{\gamma_f})^{0.5}\bigg)}{\tanh{a}}
\end{equation}
$$

From the test data, the parameters in this equation will be fitted to get $a$ and $\gamma_f$

And the final p-y curve will be in the format of

$$
\begin{equation}
\frac{p}{p_u} = \frac{\tanh{\bigg[A\cdot \bigg(\frac{y/D}{y/D_f}\bigg)^{0.5}\bigg]}}{\tanh{A}}
\end{equation}
$$

where:

$ A = 1.33+0.45\cdot a$, and 

$(y/D)_f = \gamma_f (2.5-1.2\ln(a))$

$$
\begin{equation}
\frac{y}{D} = \bigg[\tanh^{-1}\bigg({\frac{p}{p_u}\tanh(A)}\bigg)/A\bigg]^2 \cdot (\frac{y}{D})_f
\end{equation}
$$
In the code $\gamma_f$, i.e., the shear strength at the failure is assumed to be 0.15 for all OCR while $a$ varies with OCR as below:

|OCR|$\gamma_f$|$a$|
|--|--:|--:|
|$\le 2$|0.15|2.38|
|4|0.15|1.5|
|10|0.15|1.0|

Assuming the hyperbolic function for $a$ fitted from the three points proposed.

``` python

f_model_a = lambda ocr: 3.457/ocr + 0.647

```


In [ ]:
from scipy.interpolate import interp1d
def calc_y_mo(Ip, OCR, p_pu):
    gamma_f = 0.15 
    f_model_a = lambda ocr: 3.457/ocr + 0.647
    a = f_model_a(OCR)
    A = 1.33+0.45*a
    yD_f = gamma_f*(2.5-1.2*np.log(a))
    y_D = (1/A*np.arctanh(p_pu*np.tanh(A)))**2 *yD_f
    f_multiplier = interp1d(x=[2,4,10],y=[0.33,0.5,0.66],fill_value = (0.33, 0.66),bounds_error=False)
    if Ip>30:
        y_D_multiplier = f_multiplier([OCR])[0]
    else:
        y_D_multiplier  = 1.0
    return y_D*y_D_multiplier

In [ ]:
import src
import plotly.graph_objects as go
p=np.array([0.000, 0.050, 0.200, 0.300, 0.400, 0.500, 0.600, 0.700, 0.800, 0.900, 0.975, 1.000])
fig = src.geoplot.GEOPlot.get_figure()
fig.add_trace(go.Scatter(y=p, x=calc_y_mo(Ip=29,OCR=2,p_pu=p),name='OCR=2'))
fig.add_trace(go.Scatter(y=p, x=calc_y_mo(Ip=29,OCR=4,p_pu=p),name='OCR=4'))
fig.add_trace(go.Scatter(y=p, x=calc_y_mo(Ip=29,OCR=10,p_pu=p),name='OCR=10'))

In [ ]:
df = pd.DataFrame()
df['pmo_pu'] =p
df['OCR=2 & Ip>30'] = calc_y_mo(35,2,p)
df['OCR=4 & Ip>30'] = calc_y_mo(35,4,p)
df['OCR=8 & Ip>30'] = calc_y_mo(35,8,p)

In [ ]:
df

,pmo_pu,OCR=2 & Ip>30,OCR=4 & Ip>30,OCR=8 & Ip>30
0,0.000,0.000000,0.000000,0.000000
1,0.050,0.000030,0.000087,0.000150
2,0.200,0.000500,0.001421,0.002451
3,0.300,0.001163,0.003303,0.005692
4,0.400,0.002175,0.006163,0.010602
5,0.500,0.003645,0.010297,0.017668
6,0.600,0.005779,0.016243,0.027757
7,0.700,0.008981,0.025046,0.042525
8,0.800,0.014214,0.039072,0.065589
9,0.900,0.024630,0.065419,0.107095


In [ ]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
def tanh_function(x, a, b):
    return a * 1/x + b

# Three example points
y_data = np.array([2.38,1.5,1.0])
x_data = np.array([2,4,10])

# Perform the curve fit
popt, _ = curve_fit(tanh_function, x_data, y_data)
print(popt)
model_a = lambda ocr: 3.457/ocr + 0.647

[3.45714286 0.64714286]


In [ ]:
x_fit = np.linspace(min(x_data), max(x_data), 100)
y_fit = tanh_function(x_fit, *popt)

In [ ]:
# fig = px.line(y=[2.38,2.38,1.5,1.0], x=[1,2,4,10])
x_data = np.array([2,4,10])
y_data = np.array([2.38,1.5,1])
from src import geoplot
fig = geoplot.GEOPlot.get_figure()
fig.add_trace(go.Scatter(x=x_data,y=y_data,mode='markers',name='ISO Points'))
fig.add_trace(go.Scatter(x=x_data,y=model_a(x_data),mode='markers',name='ISO Points'))
fig.add_trace(go.Scatter(x=x_fit,y=y_fit,name='Fitted Hyperbolic function'))
fig.update_layout(width=800,height=400)
fig.update_xaxes(title='ORC(>2.0)')
fig.update_yaxes(title='Coefficient (a)')

In [ ]:
popt

array([3.45714286, 0.64714286])

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
def tanh_function(x, a,b):
    return a/x+b
# Three example points
x_data = np.array([2,4,10])
y_data = np.array([0.33,0.5,0.66])

# Perform the curve fit
popt, _ = curve_fit(tanh_function, x_data, y_data)
print(popt)

x_fit = np.linspace(min(x_data), max(x_data), 100)
y_fit = tanh_function(x_fit, *popt)
#
model_a = lambda ocr: 3.457/ocr + 0.647
from src import geoplot
fig = geoplot.GEOPlot.get_figure()
fig.add_trace(go.Scatter(x=x_data,y=y_data,mode='markers',name='ISO Points'))
fig.add_trace(go.Scatter(x=x_fit,y=y_fit,name='Fitted Hyperbolic function')) 
fig.update_layout(width=800,height=400)
fig.update_xaxes(title='ORC(>2.0)')
fig.update_yaxes(title='Coefficient (a)')

[-0.81020408  0.72622449]
